### Uncomment and run the following cells if you work on Google Colab :) Don't forget to change your runtime type to GPU!

In [8]:
!git clone https://github.com/kstathou/vector_engine

Cloning into 'vector_engine'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 74 (delta 14), reused 10 (delta 10), pack-reused 54 (from 1)
Receiving objects: 100% (74/74), 25.48 MiB | 27.88 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [9]:
!cd vector_engine

In [ ]:
# pip install -r requirements.txt

### Let's begin!

In [10]:
%load_ext autoreload

In [42]:
!pip uninstall fsspec





Found existing installation: fsspec 2024.12.0
Uninstalling fsspec-2024.12.0:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/fsspec-2024.12.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/fsspec/*
Proceed (Y/n)? Y
  Successfully uninstalled fsspec-2024.12.0


In [11]:
%autoreload 2
# Used to import data from S3.
import pandas as pd
import s3fs

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle
from pathlib import Path

# Used to do vector searches and display the results.
from vector_engine.utils import vector_search, id2details

ModuleNotFoundError: No module named 's3fs'

In [12]:
!pip install --upgrade torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Stored and processed data in s3

In [14]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.4 MB/s eta 0:00:00


In [4]:
# Use pandas to read files from S3 buckets!
from datasets import load_dataset

ds = load_dataset("fancyzhx/ag_news")

In [13]:
ds["train"][:10]

{'text': ["Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
  'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.',
  "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.",
  'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.',
  'Oil prices soar to all-time record, posing new menace to 

In [11]:
print(f"Misinformation, disinformation and fake news papers: {df.id.unique().shape[0]}")

AttributeError: 'DataFrame' object has no attribute 'unique'

The [Sentence Transformers library](https://github.com/UKPLab/sentence-transformers) offers pretrained transformers that produce SOTA sentence embeddings. Checkout this [spreadsheet](https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/) with all the available models.

In this tutorial, we will use the `distilbert-base-nli-stsb-mean-tokens` model which has the best performance on Semantic Textual Similarity tasks among the DistilBERT versions. Moreover, although it's slightly worse than BERT, it is quite faster thanks to having a smaller size.

I use the same model in [Orion's semantic search engine](https://www.orion-search.org/)!

In [20]:
!pip install --upgrade torch torchvision transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 112.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 145.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu124
    Uninstalling torchvision-0.20.1+cu124:
      Successfully uninstalled torchvision-0.20.1+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.18 requires torch<2.6,>=1.10, but you have torch 2.6.0 which is incompatible.


In [19]:
!pip install sentence_transformers
!pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.6 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0
    Uninstalling torch-2.6.0:
      Successfully uninstalled torch-2.6.0


In [1]:
from sentence_transformers import SentenceTransformer
import torch

In [54]:
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19


In [2]:
# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.82k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

cpu


In [5]:
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

cuda:0


In [6]:
# Convert abstracts to vectors
embeddings = model.encode(ds["train"]["text"], show_progress_bar=True)

Batches:   0%|          | 0/3750 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [8]:
!pip install semantic_text_splitter

In [25]:
import hdbscan
from sklearn.metrics.pairwise import cosine_similarity
import spacy

In [35]:
texts = ds["train"]["text"][:100]
nlp = spacy.load("en_core_web_sm")


In [36]:
all_sentences = []
for text in texts:
    doc = nlp(text)
    all_sentences.extend([sent.text for sent in doc.sents])


In [37]:
embeddings = model.encode(all_sentences, show_progress_bar=True)



Batches:   0%|          | 0/6 [00:00<?, ?it/s]

In [38]:
print(f'Shape of the vectorised abstract: {embeddings[0].shape}')

Shape of the vectorised abstract: (384,)


In [39]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=2)
cluster_labels = clusterer.fit_predict(embeddings)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [40]:
chunks = {}
for idx, label in enumerate(cluster_labels):
    if label != -1:  # Ігноруємо шумові точки
        if label not in chunks:
            chunks[label] = []
        chunks[label].append(all_sentences[idx])


In [54]:
chunks

array([[14.],
       [17.],
       [13.],
       [ 5.],
       [11.],
       [16.],
       [ 0.],
       [ 1.],
       [ 2.],
       [12.],
       [ 7.],
       [ 8.],
       [ 4.],
       [10.],
       [ 9.],
       [15.],
       [ 6.],
       [ 3.]], dtype=float32)

## Vector similarity search with Faiss
[Faiss](https://github.com/facebookresearch/faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, even ones that do not fit in RAM.
    
Faiss is built around the `Index` object which contains, and sometimes preprocesses, the searchable vectors. Faiss has a large collection of [indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes). You can even create [composite indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes-(composite)). Faiss handles collections of vectors of a fixed dimensionality d, typically a few 10s to 100s.

**Note**: Faiss uses only 32-bit floating point matrices. This means that you will have to change the data type of the input before building the index.

To learn more about Faiss, you can read their paper on [arXiv](https://arxiv.org/abs/1702.08734).

Here, we will the `IndexFlatL2` index:
- It's a simple index that performs a brute-force L2 distance search
- It scales linearly. It will work fine with our data but you might want to try [faster indexes](https://github.com/facebookresearch/faiss/wiki/Faster-search) if you work will millions of vectors.

To create an index with the `misinformation` abstract vectors, we will:
1. Change the data type of the abstract vectors to float32.
2. Build an index and pass it the dimension of the vectors it will operate on.
3. Pass the index to IndexIDMap, an object that enables us to provide a custom list of IDs for the indexed vectors.
4. Add the abstract vectors and their ID mapping to the index. In our case, we will map vectors to their paper IDs from MAG.

In [47]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 19.5 MB/s eta 0:00:00


In [48]:
import numpy as np
import faiss

In [ ]:
# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

In [49]:
chunks = np.array([chunk for chunk in chunks]).astype("float32")

In [56]:
index = faiss.IndexFlatL2(chunks.shape[1])  # This will work as chunks.shape[1] = 1

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Add vectors with IDs (assuming `ds["train"]["id"]` is an array of integers)
index.add_with_ids(chunks, np.array(range(len(chunks))))
print(f"Number of vectors in the Faiss index: {index.ntotal}")

Number of vectors in the Faiss index: 18


In [57]:
# Example query: search for the 3 nearest neighbors to a given chunk (query vector)
query_vector = np.array([[15.]], dtype=np.float32)  # This should be a 1D vector like your chunks
k = 3  # Number of neighbors to retrieve

# Perform the search
distances, indices = index.search(query_vector, k)

print(f"Distances: {distances}")
print(f"Indices: {indices}")


Distances: [[0. 1. 1.]]
Indices: [[15  0  5]]


In [52]:


# Step 2: Instantiate the index
index = faiss.IndexFlatL2(chunks.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(chunks, texts)

print(f"Number of vectors in the Faiss index: {index.ntotal}")

ValueError: invalid literal for int() with base 10: "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again."

### Searching the index
The index we built will perform a k-nearest-neighbour search. We have to provide the number of neighbours to be returned.

Let's query the index with an abstract from our dataset and retrieve the 10 most relevant documents. **The first one must be our query!**


In [30]:
# Paper abstract
ds[].iloc[5415, 1]

AttributeError: 'Dataset' object has no attribute 'iloc'

In [39]:
query = "Cats and dogs like fish"
query_embedding = model.encode([query])[0]

In [40]:
D, I = index.search(np.array([query_embedding]), k=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

L2 distance: [208.31039428710938, 210.7880401611328, 213.5262451171875, 215.68603515625, 217.42323303222656, 220.2823486328125, 220.79281616210938, 221.435546875, 228.43553161621094, 230.3131103515625]

MAG paper IDs: [29179, 116236, 30379, 29573, 63913, 28661, 73821, 112118, 27473, 63422]


In [41]:
ds["train"][29179]

{'text': 'Egyptians treated animals like kings Ancient Egyptians revered cats and other animals and took as much care in preparing them for their passage to the next life as they did with humans, scientists said Wednesday.',
 'label': 3,
 'id': 29179}

In [31]:
# Retrieve the 10 nearest neighbours
D, I = index.search(np.array([embeddings[5415]]), k=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

L2 distance: [0.0, 95.68473815917969, 98.63288879394531, 99.77638244628906, 102.4472427368164, 103.07432556152344, 116.07532501220703, 128.68209838867188, 129.59927368164062, 131.41485595703125]

MAG paper IDs: [5415, 5547, 5542, 5782, 6407, 105229, 6360, 6351, 105381, 5614]


In [33]:
ds["train"][5415]

{'text': "Pinsent bids for glory British rower Matthew Pinsent will be aiming for his fourth Olympic gold when 'Super Saturday' starts in Athens.",
 'label': 1,
 'id': 5415}

In [34]:
ds["train"][5547]

{'text': "Fourth gold for Pinsent Britain's Matthew Pinsent claims his fourth Olympic gold as the men's coxless four triumph in a photo finish.",
 'label': 1,
 'id': 5547}

In [32]:
# Fetch the paper titles based on their index
id2details(ds, I, 'original_title')

NameError: name 'id2details' is not defined

In [ ]:
# Fetch the paper abstracts based on their index
id2details(df, I, 'abstract')

[["We address the diffusion of information about the COVID-19 with a massive data analysis on Twitter, Instagram, YouTube, Reddit and Gab. We analyze engagement and interest in the COVID-19 topic and provide a differential assessment on the evolution of the discourse on a global scale for each platform and their users. We fit information spreading with epidemic models characterizing the basic reproduction number [Formula: see text] for each social media platform. Moreover, we identify information spreading from questionable sources, finding different volumes of misinformation in each platform. However, information from both reliable and questionable sources do not present different spreading patterns. Finally, we provide platform-dependent numerical estimates of rumors' amplification."],
 ["We address the diffusion of information about the COVID-19 with a massive data analysis on Twitter, Instagram, YouTube, Reddit and Gab. We analyze engagement and interest in the COVID-19 topic and p


## Putting all together

So far, we've built a Faiss index using the misinformation abstract vectors we encoded with a sentence-DistilBERT model. That's helpful but in a real case scenario, we would have to work with unseen data. To query the index with an unseen query and retrieve its most relevant documents, we would have to do the following:

1. Encode the query with the same sentence-DistilBERT model we used for the rest of the abstract vectors.
2. Change its data type to float32.
3. Search the index with the encoded query.

Here, we will use the introduction of an article published on [HKS Misinformation Review](https://misinforeview.hks.harvard.edu/article/can-whatsapp-benefit-from-debunked-fact-checked-stories-to-reduce-misinformation/).


In [ ]:
user_query = """
WhatsApp was alleged to have been widely used to spread misinformation and propaganda
during the 2018 elections in Brazil and the 2019 elections in India. Due to the
private encrypted nature of the messages on WhatsApp, it is hard to track the dissemination
of misinformation at scale. In this work, using public WhatsApp data from Brazil and India, we
observe that misinformation has been largely shared on WhatsApp public groups even after they
were already fact-checked by popular fact-checking agencies. This represents a significant portion
of misinformation spread in both Brazil and India in the groups analyzed. We posit that such
misinformation content could be prevented if WhatsApp had a means to flag already fact-checked
content. To this end, we propose an architecture that could be implemented by WhatsApp to counter
such misinformation. Our proposal respects the current end-to-end encryption architecture on WhatsApp,
thus protecting users’ privacy while providing an approach to detect the misinformation that benefits
from fact-checking efforts.
"""

In [ ]:
# For convenience, I've wrapped all steps in the vector_search function.
# It takes four arguments:
# A query, the sentence-level transformer, the Faiss index and the number of requested results
D, I = vector_search([user_query], model, index, num_results=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

L2 distance: [7.384466171264648, 57.3224983215332, 57.3224983215332, 71.48453521728516, 72.06803131103516, 79.13472747802734, 86.0128173828125, 89.91024780273438, 90.76014709472656, 90.76422119140625]

MAG paper IDs: [3047438096, 3021927925, 3037966274, 2889959140, 2791045616, 2943077655, 3014380170, 2967434249, 3028584171, 2990343632]


In [ ]:
# Fetching the paper titles based on their index
id2details(df, I, 'original_title')

[['Can WhatsApp Benefit from Debunked Fact-Checked Stories to Reduce Misinformation?'],
 ['A Dataset of Fact-Checked Images Shared on WhatsApp During the Brazilian and Indian Elections'],
 ['A Dataset of Fact-Checked Images Shared on WhatsApp During the Brazilian and Indian Elections'],
 ['A System for Monitoring Public Political Groups in WhatsApp'],
 ['Politics of Fake News: How WhatsApp Became a Potent Propaganda Tool in India'],
 ['Characterizing Attention Cascades in WhatsApp Groups'],
 ['OS IMPACTOS JURÍDICOS E SOCIAIS DAS FAKE NEWS EM TERRITÓRIO BRASILEIRO'],
 ['Fake News and Social Media: Indian Perspective'],
 ['Images and Misinformation in Political Groups: Evidence from WhatsApp in India'],
 ['Can WhatsApp Counter Misinformation by Limiting Message Forwarding']]

In [ ]:
# Define project base directory
# Change the index from 1 to 0 if you run this on Google Colab
project_dir = Path('notebooks').resolve().parents[1]
print(project_dir)

# Serialise index and store it as a pickle
with open(f"{project_dir}/models/faiss_index.pickle", "wb") as h:
    pickle.dump(faiss.serialize_index(index), h)

/content/vector_engine
